In [1]:
import pandas as pd
import pathlib
import os
import xlrd
import re
from pathlib import Path

In [209]:
### vilken mapp vill du kolla i? INF
## Skapar en lista över excelfilerna i mappen
files = os.listdir('protein-peptide_sample_inf')
files_xls = [f for f in files if f[-3:] == 'lsx']
## Läser in filerna 1 och 1 och appendar dom i df_inf
df_inf = pd.DataFrame()
for f in files_xls:
    data = pd.read_excel('protein-peptide_sample_inf' + '/' + f)
    ## sortera bort om feature<4
    data = data.loc[(data['#Feature'] >= 4)]
    ## Tar bort allt ur kolummn som inte är bokstäver + ta bort upp och nedströms AA
    data['Peptide']= data['Peptide'].str.replace('\d+', '')
    data['Peptide']= data['Peptide'].str.replace(r'\(\+\.\)', '')
    data['Peptide']= data['Peptide'].apply(lambda x: x.split('.')[1] if x[1]=='.' else x.split('.')[0] if (x[1]!='.' and x[-2]=='.') else x)
    data['count'] = 1
    ## Summerar Peptides, sparar protein info
    data = data.groupby(by=['Peptide', 'Protein Accession'], as_index=False).sum()
    data['End'] = data['End']/ data['count']
    data['Start'] = data['Start']/ data['count']
    ## ta bort duplikater om ej enstaka protein ska analyseras
    data = data.drop_duplicates('Peptide')
    df_inf = df_inf.append(data)
    
df_inf['count']=1
df_inf.reset_index(inplace=True, drop=True)

## Välja vad som ska analyseras!!
#df_inf= df_inf.loc[(df_inf['Protein Accession'] == 'sp|P68871|HBB_HUMAN')]
df_inf

,Peptide,Protein Accession,Protein Group,Protein ID,-10lgP,Mass,Length,ppm,m/z,z,...,Fraction,#Feature,#Feature Sample 31,Start,End,count,Area Sample 34,#Feature Sample 34,Area Sample 39,#Feature Sample 39
0,AAHLPAEF,sp|P69905|HBA_HUMAN,1,137976,28.23,854.4286,8,1.0,428.2220,2,...,11,4,4.0,111.0,118.0,1,NaN,NaN,NaN,NaN
1,AAHLPAEFTPA,sp|P69905|HBA_HUMAN,1,137976,50.35,1123.5662,11,0.0,562.7903,2,...,9,4,4.0,111.0,121.0,1,NaN,NaN,NaN,NaN
2,AAHLPAEFTPAV,sp|P69905|HBA_HUMAN,1,137976,49.91,1222.6345,12,3.5,612.3267,2,...,9,4,4.0,111.0,122.0,1,NaN,NaN,NaN,NaN
3,AAHLPAEFTPAVH,sp|P69905|HBA_HUMAN,1,137976,61.71,1359.6935,13,1.5,454.2391,3,...,9,7,7.0,111.0,123.0,1,NaN,NaN,NaN,NaN
4,AAHLPAEFTPAVHA,sp|P69905|HBA_HUMAN,1,137976,55.47,1430.7306,14,1.6,477.9182,3,...,9,8,8.0,111.0,124.0,1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2083,YQKVVAGV,sp|P02042|HBD_HUMAN,3,137977,38.88,862.4912,8,0.9,432.2533,2,...,57,5,NaN,131.0,138.0,1,NaN,NaN,18200000.0,5.0
2084,YQKVVAGVAN,sp|P02042|HBD_HUMAN,3,137977,40.93,1047.5713,10,3.6,524.7948,2,...,57,4,NaN,131.0,140.0,1,NaN,NaN,5510000.0,4.0
2085,YQKVVAGVANA,sp|P02042|HBD_HUMAN,3,137977,48.40,1118.6084,11,4.9,560.3142,2,...,58,4,NaN,131.0,141.0,1,NaN,NaN,4020000.0,4.0
2086,YQKVVAGVANAL,sp|P02042|HBD_HUMAN,3,137977,48.54,1231.6924,12,4.6,616.8563,2,...,57,4,NaN,131.0,142.0,1,NaN,NaN,11200000.0,4.0


In [210]:
## Välja vad som ska analyseras!!
# region=111
# df_inf= df_inf.loc[(df_inf['Start'] >= region)]

df_inf = df_inf[['Protein Accession','Start','End','Peptide', '#Feature']]
## Spara endast kort Protein Accession
df_inf['Protein Accession'] = df_inf['Protein Accession'].map(lambda x: str(x[3:]) )
df_inf['Protein Accession'] = df_inf['Protein Accession'].str[:6]
df_inf.reset_index(inplace=True, drop=True)
df_inf['count']=1
df_inf = df_inf.groupby(by=['Peptide','Protein Accession'], as_index=False).sum()
df_inf['End']=df_inf['End']/df_inf['count']
df_inf['Start']=df_inf['Start']/df_inf['count']
df_inf=df_inf.drop(columns='count')
df_inf.sort_values(['#Feature'], ascending=False)

<ipython-input-210-08fe9effd323>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inf['Protein Accession'] = df_inf['Protein Accession'].map(lambda x: str(x[3:]) )
<ipython-input-210-08fe9effd323>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inf['Protein Accession'] = df_inf['Protein Accession'].str[:6]
<ipython-input-210-08fe9effd323>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

,Peptide,Protein Accession,Start,End,#Feature
67,AGVANALAHKYH,P02042,136.0,147.0,109
520,GVANALAHKYH,P02042,137.0,147.0,109
126,ANALAHKYH,P02042,139.0,147.0,94
1262,VHLTPEEKSAV,P68871,2.0,12.0,78
1064,STVLTSKYR,P69905,134.0,142.0,77
...,...,...,...,...,...
473,GKVGAHAGEYGAE,P69905,16.0,28.0,4
471,GKEFTPPVQA,P68871,120.0,129.0,4
467,GHGKKVADALTNA,P69905,58.0,70.0,4
961,SDGVIKVF,P23528,8.0,15.0,4


In [192]:
df_inf_csv=df_inf.copy()
df_inf_csv = df_inf_csv[['Protein Accession', 'Start','End']]
df_inf_csv.to_csv('proteasix_tot-inf.csv', sep='\t')

### Analys

In [211]:
df_pro = pd.read_excel('proteasix_data.xlsx','inf_tot')
df_pro['Start'] = df_pro['Start amino acid']
df_pro['End'] = df_pro['End amino acid']
df_pro['Protein Accession'] = df_pro['Parent Protein AC from input']
df_pro = df_pro[['Protease Human/Mouse/Rat', 'Start', 'End', 'Protein Accession']]
df_pro

,Protease Human/Mouse/Rat,Start,End,Protein Accession
0,PGA3 (P0DJD8; PEPA3_HUMAN),128,147,P02766
1,PGA3 (P0DJD8; PEPA3_HUMAN),271,279,P08670
2,PGA3 (P0DJD8; PEPA3_HUMAN),271,279,P08670
3,PGA3 (P0DJD8; PEPA3_HUMAN),456,466,P08670
4,PGA3 (P0DJD8; PEPA3_HUMAN),22,33,P60709
...,...,...,...,...
21455,PCSK2 (P16519; NEC2_HUMAN). Pcsk2 (P21661; NEC...,181,193,P02730
21456,Pcsk5 (P41413; PCSK5_RAT). Pcsk5 (Q04592; PCSK...,181,193,P02730
21457,PCSK2 (P16519; NEC2_HUMAN). Pcsk2 (P21661; NEC...,9,19,P27105
21458,Pcsk5 (P41413; PCSK5_RAT). Pcsk5 (Q04592; PCSK...,9,19,P27105


In [213]:
df = pd.merge(df_inf, df_pro, on=['Protein Accession','Start','End'])
df = df[['Protease Human/Mouse/Rat', 'Start', 'End', '#Feature', 'Peptide']]
df = df.groupby(by= ['Protease Human/Mouse/Rat'], as_index=False).sum()
df = df.sort_values(['#Feature'], ascending=False)
df

,Protease Human/Mouse/Rat,Start,End,#Feature
55,PGA3 (P0DJD8; PEPA3_HUMAN),263503.0,283608.0,15880
10,Capn2 (O08529; CAN2_MOUSE). CAPN2 (P17655; CAN...,242068.0,259585.0,13071
9,Capn1 (O35350; CAN1_MOUSE). CAPN1 (P07384; CAN...,227078.0,243930.0,12716
19,Ctsl (P06797; CATL1_MOUSE). Ctsl (P07154; CATL...,195292.0,209491.0,11546
42,MMP7 (P09237; MMP7_HUMAN). Mmp7 (P50280; MMP7_...,149418.0,164224.0,10700
...,...,...,...,...
16,Ctrc (P55091; CTRC_RAT). Ctrc (Q3SYP2; CTRC_MO...,235.0,257.0,6
36,KLKB1 (P03952; KLKB1_HUMAN). Klkb1 (P14272; KL...,381.0,389.0,5
59,Pcsk4 (P29121; PCSK4_MOUSE). PCSK4 (Q6UW60; PC...,672.0,678.0,4
54,PCSK7 (Q16549; PCSK7_HUMAN). Pcsk7 (Q61139; PC...,672.0,678.0,4


In [214]:
filename='Proteasix_output.xlsx'
sheetname='tot-inf'
from openpyxl import load_workbook
with pd.ExcelWriter(filename, engine='openpyxl') as writer:
    writer.book = load_workbook(filename)
    df.to_excel(writer, sheetname)

### Non-inf

In [193]:
### vilken mapp vill du kolla i? ninf
## Skapar en lista över excelfilerna i mappen
files = os.listdir('protein-peptide_sample_ninf')
files_xls = [f for f in files if f[-3:] == 'lsx']
## Läser in filerna 1 och 1 och appendar dom i df_ninf
df_ninf = pd.DataFrame()
for f in files_xls:
    data = pd.read_excel('protein-peptide_sample_ninf' + '/' + f)
    ## sortera bort om feature<4
    data = data.loc[(data['#Feature'] >= 4)]
    ## Tar bort allt ur kolummn som inte är bokstäver + ta bort upp och nedströms AA
    data['Peptide']= data['Peptide'].str.replace('\d+', '')
    data['Peptide']= data['Peptide'].str.replace(r'\(\+\.\)', '')
    data['Peptide']= data['Peptide'].apply(lambda x: x.split('.')[1] if x[1]=='.' else x.split('.')[0] if (x[1]!='.' and x[-2]=='.') else x)
    data['count'] = 1
    ## Summerar Peptides, sparar protein ninfo
    data = data.groupby(by=['Peptide', 'Protein Accession'], as_index=False).sum()
    data['End'] = data['End']/ data['count']
    data['Start'] = data['Start']/ data['count']
    ## ta bort duplikater om ej enstaka protein ska analyseras
    data = data.drop_duplicates('Peptide')
    df_ninf = df_ninf.append(data)
    
df_ninf['count']=1
df_ninf.reset_index(inplace=True, drop=True)
# df_ninf= df_ninf.loc[(df_ninf['Protein Accession'] == 'sp|P68871|HBB_HUMAN')]
df_ninf

,Peptide,Protein Accession,Protein Group,Protein ID,-10lgP,Mass,Length,ppm,m/z,z,...,Fraction,#Feature,#Feature Sample 13,Start,End,count,Area Sample 21,#Feature Sample 21,Area Sample 33,#Feature Sample 33
0,AAHLPAEFTPAVHA,sp|P69905|HBA_HUMAN,1,137976,46.78,1430.7306,14,0.3,477.9176,3,...,3,4,4.0,111.0,124.0,1,NaN,NaN,NaN,NaN
1,AAWGKVGAHAGEYGAEALE,sp|P69905|HBA_HUMAN,1,137976,36.59,1885.8958,19,0.4,629.6395,3,...,3,4,4.0,13.0,31.0,1,NaN,NaN,NaN,NaN
2,AAWGKVGAHAGEYGAEALER,sp|P69905|HBA_HUMAN,1,137976,56.47,2041.9969,20,2.6,681.6747,3,...,1,7,7.0,13.0,32.0,1,NaN,NaN,NaN,NaN
3,ADALTNAVAHV,sp|P69905|HBA_HUMAN,1,137976,38.91,1080.5564,11,0.2,541.2856,2,...,1,4,4.0,64.0,74.0,1,NaN,NaN,NaN,NaN
4,ADSGEGDFLAEGGGVR,sp|P02671|FIBA_HUMAN,2,137979,58.48,1535.6852,16,0.3,768.8501,2,...,2,9,9.0,20.0,35.0,1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1438,YGTGSETESPR,sp|P02671|FIBA_HUMAN,6,137979,46.05,1182.5153,11,0.1,592.2650,2,...,35,4,NaN,277.0,287.0,1,NaN,NaN,686000.0,4.0
1439,YGTGSETESPRNPS,sp|P02671|FIBA_HUMAN,6,137979,46.84,1480.6429,14,2.0,741.3302,2,...,33,4,NaN,277.0,290.0,1,NaN,NaN,502000.0,4.0
1440,YLPAVDEKL,sp|P02655|APOC2_HUMAN,82,138134,34.10,1046.5648,9,1.0,524.2902,2,...,33,4,NaN,63.0,71.0,1,NaN,NaN,592000.0,4.0
1441,YLYEIAR,sp|P02768|ALBU_HUMAN,11,137987,31.57,926.4861,7,-0.1,464.2503,2,...,35,4,NaN,162.0,168.0,1,NaN,NaN,21700000.0,4.0


In [194]:
## VÄLJA REGION
# region=111
# df_ninf= df_ninf.loc[(df_ninf['Start'] >= region)]
df_ninf = df_ninf[['Protein Accession','Start','End','Peptide', '#Feature']]
## Spara endast kort Protein Accession
df_ninf['Protein Accession'] = df_ninf['Protein Accession'].map(lambda x: str(x[3:]) )
df_ninf['Protein Accession'] = df_ninf['Protein Accession'].str[:6]
df_ninf.reset_index(inplace=True, drop=True)
df_ninf['count']=1
df_ninf = df_ninf.groupby(by=['Peptide','Protein Accession'], as_index=False).sum()
df_ninf['End']=df_ninf['End']/df_ninf['count']
df_ninf['Start']=df_ninf['Start']/df_ninf['count']
df_ninf=df_ninf.drop(columns='count')
df_ninf

<ipython-input-194-c95a2fe5474a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ninf['Protein Accession'] = df_ninf['Protein Accession'].map(lambda x: str(x[3:]) )
<ipython-input-194-c95a2fe5474a>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ninf['Protein Accession'] = df_ninf['Protein Accession'].str[:6]
<ipython-input-194-c95a2fe5474a>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

,Peptide,Protein Accession,Start,End,#Feature
0,AAHLPAEFTPA,P69905,111.0,121.0,4
1,AAHLPAEFTPAV,P69905,111.0,122.0,4
2,AAHLPAEFTPAVH,P69905,111.0,123.0,4
3,AAHLPAEFTPAVHA,P69905,111.0,124.0,10
4,AAHLPAEFTPAVHASLDKF,P69905,111.0,129.0,6
...,...,...,...,...,...
1032,YLPAVDEKL,P02655,63.0,71.0,4
1033,YLYEIAR,P02768,162.0,168.0,12
1034,YPWTQRF,P02042,36.0,42.0,6
1035,YQKVVAGVANALAHKYH,P02042,131.0,147.0,4


In [195]:
df_ninf_csv=df_ninf.copy()
df_ninf_csv = df_ninf_csv[['Protein Accession', 'Start','End']]
df_ninf_csv.to_csv('proteasix_tot-ninf.csv', sep='\t')

### ANALYS

In [216]:
df_pro = pd.read_excel('proteasix_data.xlsx','ninf_tot')
df_pro['Start'] = df_pro['Start amino acid']
df_pro['End'] = df_pro['End amino acid']
df_pro['Protein Accession'] = df_pro['Parent Protein AC from input']
df_pro = df_pro[['Protease Human/Mouse/Rat', 'Start', 'End', 'Protein Accession']]
df_pro

,Protease Human/Mouse/Rat,Start,End,Protein Accession
0,PGA3 (P0DJD8; PEPA3_HUMAN),179,190,P60709
1,CTSD (P07339; CATD_HUMAN). Ctsd (P18242; CATD_...,179,190,P60709
2,PGA3 (P0DJD8; PEPA3_HUMAN),179,190,P60709
3,PGA3 (P0DJD8; PEPA3_HUMAN),51,62,P60709
4,PGA3 (P0DJD8; PEPA3_HUMAN),286,294,P60709
...,...,...,...,...
17689,PCSK6 (P29122; PCSK6_HUMAN). Pcsk6 (Q63415; PC...,542,554,P11171
17690,Pcsk5 (P41413; PCSK5_RAT). Pcsk5 (Q04592; PCSK...,542,554,P11171
17691,PCSK7 (Q16549; PCSK7_HUMAN). Pcsk7 (Q61139; PC...,542,554,P11171
17692,PCSK2 (P16519; NEC2_HUMAN). Pcsk2 (P21661; NEC...,38,49,P13647


In [217]:
df = pd.merge(df_ninf, df_pro, on=['Protein Accession','Start','End'])
df = df[['Protease Human/Mouse/Rat', 'Start', 'End', '#Feature', 'Peptide']]
df = df.groupby(by= ['Protease Human/Mouse/Rat'], as_index=False).sum()
df = df.sort_values(['#Feature'], ascending=False)
df

,Protease Human/Mouse/Rat,Start,End,#Feature
11,Capn2 (O08529; CAN2_MOUSE). CAPN2 (P17655; CAN...,857508.0,871297.0,9286
10,Capn1 (O35350; CAN1_MOUSE). CAPN1 (P07384; CAN...,853788.0,867669.0,9094
60,PGA3 (P0DJD8; PEPA3_HUMAN),589713.0,601871.0,8203
45,MMP7 (P09237; MMP7_HUMAN). Mmp7 (P50280; MMP7_...,719086.0,730753.0,7657
46,MMP9 (P14780; MMP9_HUMAN). Mmp9 (P41245; MMP9_...,665612.0,675396.0,6376
...,...,...,...,...
52,Mmp17 (Q9R0S3; MMP17_MOUSE). MMP17 (Q9ULZ9; MM...,824.0,843.0,8
63,PRSS3 (P35030; TRY3_HUMAN),908.0,916.0,4
41,MMP10 (P09238; MMP10_HUMAN),2.0,13.0,4
2,ADAM8 (P78325; ADAM8_HUMAN). Adam8 (Q05910; AD...,169.0,176.0,4


In [218]:
filename='Proteasix_output.xlsx'
sheetname='tot-ninf'
from openpyxl import load_workbook
with pd.ExcelWriter(filename, engine='openpyxl') as writer:
    writer.book = load_workbook(filename)
    df.to_excel(writer, sheetname)

### WF

In [196]:
### vilken mapp vill du kolla i? wf
## Skapar en lista över excelfilerna i mappen
files = os.listdir('protein-peptide_sample_wf')
files_xls = [f for f in files if f[-3:] == 'lsx']
## Läser in filerna 1 och 1 och appendar dom i df_wf
df_wf = pd.DataFrame()
for f in files_xls:
    data = pd.read_excel('protein-peptide_sample_wf' + '/' + f)
    ## sortera bort om feature<4
    data = data.loc[(data['#Feature'] >= 4)]
    ## Tar bort allt ur kolummn som inte är bokstäver + ta bort upp och nedströms AA
    data['Peptide']= data['Peptide'].str.replace('\d+', '')
    data['Peptide']= data['Peptide'].str.replace(r'\(\+\.\)', '')
    data['Peptide']= data['Peptide'].apply(lambda x: x.split('.')[1] if x[1]=='.' else x.split('.')[0] if (x[1]!='.' and x[-2]=='.') else x)
    data['count'] = 1
    ## Summerar Peptides, sparar protein wfo
    data = data.groupby(by=['Peptide', 'Protein Accession'], as_index=False).sum()
    data['End'] = data['End']/ data['count']
    data['Start'] = data['Start']/ data['count']
    ## ta bort duplikater om ej enstaka protein ska analyseras
    data = data.drop_duplicates('Peptide')
    df_wf = df_wf.append(data)
    
df_wf['count']=1
df_wf.reset_index(inplace=True, drop=True)
# df_wf= df_wf.loc[(df_wf['Protein Accession'] == 'sp|P68871|HBB_HUMAN')]
df_wf

,Peptide,Protein Accession,Protein Group,Protein ID,-10lgP,Mass,Length,ppm,m/z,z,...,End,count,Area WF2,#Feature WF2,Area WF 3,#Feature WF 3,Area WF4,#Feature WF4,Area WF5,#Feature WF5
0,AAGSDVDIDMAFILDSAETT,sp|P12111|CO6A3_HUMAN,143,308466,61.04,2056.9146,20,4.6,1029.4692,2,...,2630.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AALLSPYSYSTTAVVTNPKE,sp|P02766|TTHY_HUMAN,46,308337,85.56,2111.0786,20,2.0,1056.5487,2,...,147.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AAMSTYTGIFTDQVLSVLKGEE,sp|P02655|APOC2_HUMAN,20,308308,85.25,2375.1565,22,2.2,1188.5881,2,...,101.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AASQAALGL,sp|P02768|ALBU_HUMAN,32,308305,49.02,800.4392,9,0.0,401.2269,2,...,609.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AAWIDSVLA,sp|P00746|CFAD_HUMAN,132,308452,47.32,944.4967,9,0.7,473.2560,2,...,253.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3855,YPWTQRF,sp|P02042|HBD_HUMAN,11,308265,45.51,996.4818,7,1.6,499.2490,2,...,42.0,1,NaN,NaN,NaN,NaN,NaN,NaN,22500000.0,4.0
3856,YQGAIRQIDDIDV,sp|P04114|APOB_HUMAN,4,308260,51.08,1504.7521,13,5.5,753.3875,2,...,4128.0,1,NaN,NaN,NaN,NaN,NaN,NaN,1850000.0,4.0
3857,YQKSTELLI,sp|Q71DI3|H32_HUMAN,66,308339,41.45,1093.6019,9,3.0,547.8099,2,...,63.0,1,NaN,NaN,NaN,NaN,NaN,NaN,1800000.0,4.0
3858,YQKVVAGVANALAHKYH,sp|P02042|HBD_HUMAN,11,308265,58.64,1868.0056,17,1.8,623.6769,3,...,147.0,1,NaN,NaN,NaN,NaN,NaN,NaN,58100000.0,5.0


In [197]:
## VÄLJ REGION
# region=111
# df_wf= df_wf.loc[(df_wf['Start'] >= region)]

df_wf = df_wf[['Protein Accession','Start','End','Peptide', '#Feature']]
## Spara endast kort Protein Accession
df_wf['Protein Accession'] = df_wf['Protein Accession'].map(lambda x: str(x[3:]) )
df_wf['Protein Accession'] = df_wf['Protein Accession'].str[:6]
df_wf.reset_index(inplace=True, drop=True)
df_wf['count']=1
df_wf = df_wf.groupby(by=['Peptide','Protein Accession'], as_index=False).sum()
df_wf['End']=df_wf['End']/df_wf['count']
df_wf['Start']=df_wf['Start']/df_wf['count']
df_wf=df_wf.drop(columns='count')
df_wf

<ipython-input-197-88fe6508c9f6>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_wf['Protein Accession'] = df_wf['Protein Accession'].map(lambda x: str(x[3:]) )
<ipython-input-197-88fe6508c9f6>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_wf['Protein Accession'] = df_wf['Protein Accession'].str[:6]
<ipython-input-197-88fe6508c9f6>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

,Peptide,Protein Accession,Start,End,#Feature
0,AAFFDTASTGKTFPGFFSPML,P02671,517.0,537.0,5
1,AAGSDVDIDMAFILDSAETT,P12111,2611.0,2630.0,8
2,AALLSPYSYSTTAVVTNPKE,P02766,128.0,147.0,36
3,AAMSTYTGIFTDQVLSVLKGEE,P02655,80.0,101.0,27
4,AASQAALGL,P02768,601.0,609.0,16
...,...,...,...,...,...
1902,YVNKEIQNAV,P10909,41.0,50.0,4
1903,YVNKEIQNAVNGV,P10909,41.0,53.0,4
1904,YVNKEIQNAVNGVKQI,P10909,41.0,56.0,11
1905,YYLQGAKIPKPEA,Q14624,627.0,639.0,10


In [198]:
df_wf_csv=df_wf.copy()
df_wf_csv = df_wf_csv[['Protein Accession', 'Start','End']]
df_wf_csv.to_csv('proteasix_tot-wf.csv', sep='\t')

### ANALYS

In [219]:
df_pro = pd.read_excel('proteasix_data.xlsx','wf_tot')
df_pro['Start'] = df_pro['Start amino acid']
df_pro['End'] = df_pro['End amino acid']
df_pro['Protein Accession'] = df_pro['Parent Protein AC from input']
df_pro = df_pro[['Protease Human/Mouse/Rat', 'Start', 'End', 'Protein Accession']]
df_pro

,Protease Human/Mouse/Rat,Start,End,Protein Accession
0,PGA3 (P0DJD8; PEPA3_HUMAN),513,526,P00734
1,PGA3 (P0DJD8; PEPA3_HUMAN),319,327,P00734
2,PGA3 (P0DJD8; PEPA3_HUMAN),474,481,P00734
3,PGA3 (P0DJD8; PEPA3_HUMAN),516,526,P00734
4,PGA3 (P0DJD8; PEPA3_HUMAN),364,379,P00734
...,...,...,...,...
29254,PCSK2 (P16519; NEC2_HUMAN). Pcsk2 (P21661; NEC...,1353,1366,P0C0L4
29255,Pcsk5 (P41413; PCSK5_RAT). Pcsk5 (Q04592; PCSK...,16,25,O75367
29256,PCSK2 (P16519; NEC2_HUMAN). Pcsk2 (P21661; NEC...,2,12,P14174
29257,PCSK2 (P16519; NEC2_HUMAN). Pcsk2 (P21661; NEC...,28,41,P16949


In [220]:
df = pd.merge(df_wf, df_pro, on=['Protein Accession','Start','End'])
df = df[['Protease Human/Mouse/Rat', 'Start', 'End', '#Feature', 'Peptide']]
df = df.groupby(by= ['Protease Human/Mouse/Rat'], as_index=False).sum()
df = df.sort_values(['#Feature'], ascending=False)
df

,Protease Human/Mouse/Rat,Start,End,#Feature
61,PGA3 (P0DJD8; PEPA3_HUMAN),1041335.0,1065697.0,19988
10,Capn2 (O08529; CAN2_MOUSE). CAPN2 (P17655; CAN...,1009364.0,1031550.0,18314
9,Capn1 (O35350; CAN1_MOUSE). CAPN1 (P07384; CAN...,943231.0,965038.0,17824
45,MMP7 (P09237; MMP7_HUMAN). Mmp7 (P50280; MMP7_...,887106.0,907291.0,16066
20,Ctsl (P06797; CATL1_MOUSE). Ctsl (P07154; CATL...,789749.0,807246.0,15312
...,...,...,...,...
73,Thop1 (P24155; THOP1_RAT). THOP1 (P52888; THOP...,666.0,709.0,12
57,Nln (P42676; NEUL_RAT). Nln (Q91YP2; NEUL_MOUS...,228.0,259.0,8
53,Mmp17 (Q9R0S3; MMP17_MOUSE). MMP17 (Q9ULZ9; MM...,185.0,195.0,4
14,Casp8 (O89110; CASP8_MOUSE). CASP8 (Q14790; CA...,332.0,349.0,4


In [221]:
filename='Proteasix_output.xlsx'
sheetname='tot-wf'
from openpyxl import load_workbook
with pd.ExcelWriter(filename, engine='openpyxl') as writer:
    writer.book = load_workbook(filename)
    df.to_excel(writer, sheetname)

### DIFFERENCE INF

In [287]:
df_diff = pd.read_excel('difference_data.xlsx', 'inf_diff')
## Spara endast kort Protein Accession
df_diff['Protein Accession'] = df_diff['Accession']
df_diff['Protein Accession'] = df_diff['Protein Accession'].map(lambda x: str(x[3:]) )
df_diff['Protein Accession'] = df_diff['Protein Accession'].str[:6]
df_diff.reset_index(inplace=True, drop=True)
df_diff

,Unnamed: 0,Accession,Peptide,#Feature,#Feature Sample 13,#Feature Sample 21,#Feature Sample 33,count,#Feature Sample 31,#Feature Sample 34,#Feature Sample 39,Fold Change,Difference (ninf-inf),std ninf,std inf,Protein Accession
0,0,sp|P68871|HBB_HUMAN:sp|P02042|HBD_HUMAN,AGVANALAHKYH,150,11,12,18,6,37,12,60,2.658537,-22.666667,3.785939,24.006943,P68871
1,1,sp|P68871|HBB_HUMAN:sp|P02042|HBD_HUMAN,GVANALAHKYH,152,13,13,17,6,42,12,55,2.534884,-22.000000,2.309401,22.052967,P68871
2,2,sp|P68871|HBB_HUMAN,SDGLAHLDNLKGTFA,56,0,0,0,3,22,6,28,inf,-18.666667,0.000000,11.372481,P68871
3,3,sp|P68871|HBB_HUMAN,VHLTPEEKSAV,102,8,8,8,6,28,14,36,3.250000,-18.000000,0.000000,11.135529,P68871
4,4,sp|P69905|HBA_HUMAN,AHVDDMPNALSAL,54,0,0,0,3,22,13,19,inf,-18.000000,0.000000,4.582576,P69905
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,70,sp|P02647|APOA1_HUMAN,SALEEYTKKLNTQ,15,0,0,0,3,5,4,6,inf,-5.000000,0.000000,1.000000,P02647
71,71,sp|P02647|APOA1_HUMAN,SEKAKPALEDLR,15,0,0,0,3,5,5,5,inf,-5.000000,0.000000,0.000000,P02647
72,72,sp|P69905|HBA_HUMAN,FLSFPTTKT,15,0,0,0,3,5,4,6,inf,-5.000000,0.000000,1.000000,P69905
73,73,sp|P69905|HBA_HUMAN,SHGSAQVK,15,0,0,0,3,4,4,7,inf,-5.000000,0.000000,1.732051,P69905


In [288]:
df_inf

,Peptide,Protein Accession,Start,End,#Feature
0,AAHLPAEF,P69905,111.0,118.0,4
1,AAHLPAEFTPA,P69905,111.0,121.0,8
2,AAHLPAEFTPAV,P69905,111.0,122.0,12
3,AAHLPAEFTPAVH,P69905,111.0,123.0,19
4,AAHLPAEFTPAVHA,P69905,111.0,124.0,23
...,...,...,...,...,...
1402,YQKVVAGVAN,P02042,131.0,140.0,4
1403,YQKVVAGVANA,P02042,131.0,141.0,4
1404,YQKVVAGVANAL,P02042,131.0,142.0,8
1405,YQKVVAGVANALA,P02042,131.0,143.0,4


In [289]:
df_diff = pd.merge(df_diff, df_inf, on=['Peptide'])
# df ['Start'] = df ['Start_x']
# df ['End'] = df ['End_x']
df_diff['Protein Accession']= df_diff['Protein Accession_x']
df_diff['#Feature'] = df_diff['#Feature_y']
df_diff

,Unnamed: 0,Accession,Peptide,#Feature_x,#Feature Sample 13,#Feature Sample 21,#Feature Sample 33,count,#Feature Sample 31,#Feature Sample 34,...,Difference (ninf-inf),std ninf,std inf,Protein Accession_x,Protein Accession_y,Start,End,#Feature_y,Protein Accession,#Feature
0,0,sp|P68871|HBB_HUMAN:sp|P02042|HBD_HUMAN,AGVANALAHKYH,150,11,12,18,6,37,12,...,-22.666667,3.785939,24.006943,P68871,P02042,136.0,147.0,109,P68871,109
1,1,sp|P68871|HBB_HUMAN:sp|P02042|HBD_HUMAN,GVANALAHKYH,152,13,13,17,6,42,12,...,-22.000000,2.309401,22.052967,P68871,P02042,137.0,147.0,109,P68871,109
2,2,sp|P68871|HBB_HUMAN,SDGLAHLDNLKGTFA,56,0,0,0,3,22,6,...,-18.666667,0.000000,11.372481,P68871,P68871,73.0,87.0,56,P68871,56
3,3,sp|P68871|HBB_HUMAN,VHLTPEEKSAV,102,8,8,8,6,28,14,...,-18.000000,0.000000,11.135529,P68871,P68871,2.0,12.0,78,P68871,78
4,4,sp|P69905|HBA_HUMAN,AHVDDMPNALSAL,54,0,0,0,3,22,13,...,-18.000000,0.000000,4.582576,P69905,P69905,72.0,84.0,54,P69905,54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,70,sp|P02647|APOA1_HUMAN,SALEEYTKKLNTQ,15,0,0,0,3,5,4,...,-5.000000,0.000000,1.000000,P02647,P02647,255.0,267.0,15,P02647,15
70,71,sp|P02647|APOA1_HUMAN,SEKAKPALEDLR,15,0,0,0,3,5,5,...,-5.000000,0.000000,0.000000,P02647,P02647,228.0,239.0,15,P02647,15
71,72,sp|P69905|HBA_HUMAN,FLSFPTTKT,15,0,0,0,3,5,4,...,-5.000000,0.000000,1.000000,P69905,P69905,34.0,42.0,15,P69905,15
72,73,sp|P69905|HBA_HUMAN,SHGSAQVK,15,0,0,0,3,4,4,...,-5.000000,0.000000,1.732051,P69905,P69905,50.0,57.0,15,P69905,15


In [238]:
df_diff_csv=df_diff.copy()
df_diff_csv = df_diff_csv[['Protein Accession', 'Start','End']]
df_diff_csv.to_csv('proteasix_diff-inf.csv', sep='\t')

### Analys

In [290]:
df_pro = pd.read_excel('proteasix_data.xlsx','diff_inf')
df_pro['Start'] = df_pro['Start amino acid']
df_pro['End'] = df_pro['End amino acid']
df_pro['Protein Accession'] = df_pro['Parent Protein AC from input']
df_pro = df_pro[['Protease Human/Mouse/Rat', 'Start', 'End', 'Protein Accession']]
df_pro

,Protease Human/Mouse/Rat,Start,End,Protein Accession
0,Ctsl (P06797; CATL1_MOUSE). Ctsl (P07154; CATL...,549.0,558.0,P02768
1,Ctss (O70370; CATS_MOUSE). CTSS (P25774; CATS_...,549.0,558.0,P02768
2,Ctsk (O35186; CATK_RAT). CTSK (P43235; CATK_HU...,549.0,558.0,P02768
3,Capn1 (O35350; CAN1_MOUSE). CAPN1 (P07384; CAN...,549.0,558.0,P02768
4,Capn2 (O08529; CAN2_MOUSE). CAPN2 (P17655; CAN...,549.0,558.0,P02768
...,...,...,...,...
1080,KLK4 (Q9Y5K2; KLK4_HUMAN). Klk4 (Q9Z0M1; KLK4_...,2.0,9.0,P68871
1081,NaN,NaN,NaN,NaN
1082,NaN,NaN,NaN,NaN
1083,NaN,NaN,NaN,NaN


In [291]:
df = pd.merge(df_diff, df_pro, on=['Protein Accession','Start','End'])
df = df[['Protease Human/Mouse/Rat', 'Start', 'End', '#Feature', 'Peptide', 'Difference (ninf-inf)']]

df = df.groupby(['Protease Human/Mouse/Rat'], as_index=False).sum()

df = df.sort_values(['#Feature'], ascending=False)
df

,Protease Human/Mouse/Rat,Start,End,#Feature,Difference (ninf-inf)
39,PGA3 (P0DJD8; PEPA3_HUMAN),7117.0,8027.0,2974,-814.000000
16,ELANE (P08246; ELNE_HUMAN). Elane (Q3UP87; ELN...,6497.0,7192.0,2237,-637.666667
6,Capn1 (O35350; CAN1_MOUSE). CAPN1 (P07384; CAN...,7166.0,7869.0,2229,-605.000000
7,Capn2 (O08529; CAN2_MOUSE). CAPN2 (P17655; CAN...,7169.0,7872.0,2215,-590.333333
14,Ctsl (P06797; CATL1_MOUSE). Ctsl (P07154; CATL...,6204.0,6878.0,2116,-612.000000
30,MMP9 (P14780; MMP9_HUMAN). Mmp9 (P41245; MMP9_...,6086.0,6753.0,1928,-556.666667
15,Ctss (O70370; CATS_MOUSE). CTSS (P25774; CATS_...,4754.0,5302.0,1774,-506.333333
5,CTSG (P08311; CATG_HUMAN). Ctsg (P17977; CATG_...,5132.0,5692.0,1611,-455.666667
32,Mep1a (P28825; MEP1A_MOUSE). MEP1A (Q16819; ME...,4349.0,4878.0,1597,-482.666667
29,MMP7 (P09237; MMP7_HUMAN). Mmp7 (P50280; MMP7_...,5046.0,5625.0,1484,-430.000000


In [292]:
filename='Proteasix_output.xlsx'
sheetname='diff_inf'
from openpyxl import load_workbook
with pd.ExcelWriter(filename, engine='openpyxl') as writer:
    writer.book = load_workbook(filename)
    df.to_excel(writer, sheetname)

### DIfference NINF

In [274]:
df_diff = pd.read_excel('difference_data.xlsx', 'ninf_diff')
## Spara endast kort Protein Accession
df_diff['Protein Accession'] = df_diff['Accession']
df_diff['Protein Accession'] = df_diff['Protein Accession'].map(lambda x: str(x[3:]) )
df_diff['Protein Accession'] = df_diff['Protein Accession'].str[:6]
df_diff.reset_index(inplace=True, drop=True)
df_diff

,Unnamed: 0,Accession,Peptide,#Feature,#Feature Sample 13,#Feature Sample 21,#Feature Sample 33,count,#Feature Sample 31,#Feature Sample 34,#Feature Sample 39,Fold Change,Difference (ninf-inf),std ninf,std inf,Protein Accession
0,166,sp|P02671|FIBA_HUMAN,EGDFLAEGGGVR,16,8,4,4,3,0,0,0,0,5.333333,2.309401,0,P02671
1,167,sp|P02671|FIBA_HUMAN,MADEAGSEADHEGTHSTKR,16,5,6,5,3,0,0,0,0,5.333333,0.577350,0,P02671
2,168,sp|P69905|HBA_HUMAN,FPHFDLSHG,16,5,5,6,3,0,0,0,0,5.333333,0.577350,0,P69905
3,169,sp|P02671|FIBA_HUMAN,GDFLAEGGGVR,16,6,6,4,3,0,0,0,0,5.333333,1.154701,0,P02671
4,170,sp|P68871|HBB_HUMAN:sp|P02042|HBD_HUMAN,VHLTPEE,17,6,6,5,3,0,0,0,0,5.666667,0.577350,0,P68871
5,171,sp|Q14624|ITIH4_HUMAN,QLGLPGPPDVPDHAAYHPFR,18,7,7,4,3,0,0,0,0,6.000000,1.732051,0,Q14624
6,172,sp|P02671|FIBA_HUMAN,DEAGSEADHEGTHST,18,5,6,7,3,0,0,0,0,6.000000,1.000000,0,P02671
7,173,sp|P01024|CO3_HUMAN,TLDPERLGR,19,5,8,5,3,0,0,0,0,6.000000,1.732051,0,P01024
8,174,sp|P69905|HBA_HUMAN,SALSDLHAH,18,6,6,6,3,0,0,0,0,6.000000,0.000000,0,P69905
9,175,sp|P69905|HBA_HUMAN,YFPHFDLSHGSAQVK,19,7,8,4,3,0,0,0,0,6.333333,2.081666,0,P69905


In [275]:
df_ninf

,Peptide,Protein Accession,Start,End,#Feature
0,AAHLPAEFTPA,P69905,111.0,121.0,4
1,AAHLPAEFTPAV,P69905,111.0,122.0,4
2,AAHLPAEFTPAVH,P69905,111.0,123.0,4
3,AAHLPAEFTPAVHA,P69905,111.0,124.0,10
4,AAHLPAEFTPAVHASLDKF,P69905,111.0,129.0,6
...,...,...,...,...,...
1032,YLPAVDEKL,P02655,63.0,71.0,4
1033,YLYEIAR,P02768,162.0,168.0,12
1034,YPWTQRF,P02042,36.0,42.0,6
1035,YQKVVAGVANALAHKYH,P02042,131.0,147.0,4


In [276]:
df_diff = pd.merge(df_diff, df_ninf, on=['Peptide'])
# df ['Start'] = df ['Start_x']
# df ['End'] = df ['End_x']
df_diff['Protein Accession']= df_diff['Protein Accession_x']
df_diff['#Feature'] = df_diff['#Feature_y']
df_diff

,Unnamed: 0,Accession,Peptide,#Feature_x,#Feature Sample 13,#Feature Sample 21,#Feature Sample 33,count,#Feature Sample 31,#Feature Sample 34,...,Difference (ninf-inf),std ninf,std inf,Protein Accession_x,Protein Accession_y,Start,End,#Feature_y,Protein Accession,#Feature
0,166,sp|P02671|FIBA_HUMAN,EGDFLAEGGGVR,16,8,4,4,3,0,0,...,5.333333,2.309401,0,P02671,P02671,24.0,35.0,16,P02671,16
1,167,sp|P02671|FIBA_HUMAN,MADEAGSEADHEGTHSTKR,16,5,6,5,3,0,0,...,5.333333,0.577350,0,P02671,P02671,603.0,621.0,16,P02671,16
2,168,sp|P69905|HBA_HUMAN,FPHFDLSHG,16,5,5,6,3,0,0,...,5.333333,0.577350,0,P69905,P69905,44.0,52.0,16,P69905,16
3,169,sp|P02671|FIBA_HUMAN,GDFLAEGGGVR,16,6,6,4,3,0,0,...,5.333333,1.154701,0,P02671,P02671,25.0,35.0,16,P02671,16
4,170,sp|P68871|HBB_HUMAN:sp|P02042|HBD_HUMAN,VHLTPEE,17,6,6,5,3,0,0,...,5.666667,0.577350,0,P68871,P02042,2.0,8.0,17,P68871,17
5,171,sp|Q14624|ITIH4_HUMAN,QLGLPGPPDVPDHAAYHPFR,18,7,7,4,3,0,0,...,6.000000,1.732051,0,Q14624,Q14624,669.0,688.0,18,Q14624,18
6,172,sp|P02671|FIBA_HUMAN,DEAGSEADHEGTHST,18,5,6,7,3,0,0,...,6.000000,1.000000,0,P02671,P02671,605.0,619.0,18,P02671,18
7,173,sp|P01024|CO3_HUMAN,TLDPERLGR,19,5,8,5,3,0,0,...,6.000000,1.732051,0,P01024,P01024,946.0,954.0,19,P01024,19
8,174,sp|P69905|HBA_HUMAN,SALSDLHAH,18,6,6,6,3,0,0,...,6.000000,0.000000,0,P69905,P69905,82.0,90.0,18,P69905,18
9,175,sp|P69905|HBA_HUMAN,YFPHFDLSHGSAQVK,19,7,8,4,3,0,0,...,6.333333,2.081666,0,P69905,P69905,43.0,57.0,19,P69905,19


In [277]:
df_diff_csv=df_diff.copy()
df_diff_csv = df_diff_csv[['Protein Accession', 'Start','End']]
df_diff_csv.to_csv('proteasix_diff-ninf.csv', sep='\t')

In [278]:
df_pro = pd.read_excel('proteasix_data.xlsx','diff_ninf')
df_pro['Start'] = df_pro['Start amino acid']
df_pro['End'] = df_pro['End amino acid']
df_pro['Protein Accession'] = df_pro['Parent Protein AC from input']
df_pro = df_pro[['Protease Human/Mouse/Rat', 'Start', 'End', 'Protein Accession']]
df_pro

,Protease Human/Mouse/Rat,Start,End,Protein Accession
0,PGA3 (P0DJD8; PEPA3_HUMAN),44.0,52.0,P69905
1,PGA3 (P0DJD8; PEPA3_HUMAN),82.0,90.0,P69905
2,CTSD (P07339; CATD_HUMAN). Ctsd (P18242; CATD_...,82.0,90.0,P69905
3,PGA3 (P0DJD8; PEPA3_HUMAN),65.0,80.0,P69905
4,CTSD (P07339; CATD_HUMAN). Ctsd (P18242; CATD_...,65.0,80.0,P69905
...,...,...,...,...
610,PCSK2 (P16519; NEC2_HUMAN). Pcsk2 (P21661; NEC...,669.0,688.0,Q14624
611,NaN,NaN,NaN,NaN
612,NaN,NaN,NaN,NaN
613,NaN,NaN,NaN,NaN


In [284]:
df = pd.merge(df_diff, df_pro, on=['Protein Accession','Start','End'])
df = df[['Protease Human/Mouse/Rat', 'Start', 'End', '#Feature', 'Peptide', 'Difference (ninf-inf)']]

df = df.groupby(['Protease Human/Mouse/Rat'], as_index=False).sum()

df = df.sort_values(['#Feature'], ascending=False)
df

,Protease Human/Mouse/Rat,Start,End,#Feature,Difference (ninf-inf)
6,Capn2 (O08529; CAN2_MOUSE). CAPN2 (P17655; CAN...,8340.0,8919.0,1241,410.333333
5,Capn1 (O35350; CAN1_MOUSE). CAPN1 (P07384; CAN...,8052.0,8644.0,1130,374.000000
28,MMP7 (P09237; MMP7_HUMAN). Mmp7 (P50280; MMP7_...,8205.0,8706.0,1077,357.333333
39,PGA3 (P0DJD8; PEPA3_HUMAN),6407.0,6884.0,1014,336.000000
11,Ctss (O70370; CATS_MOUSE). CTSS (P25774; CATS_...,6143.0,6529.0,811,268.666667
31,Mep1a (P28825; MEP1A_MOUSE). MEP1A (Q16819; ME...,6543.0,6890.0,734,243.000000
8,Ctsb (P00787; CATB_RAT). CTSB (P07858; CATB_HU...,4723.0,5045.0,708,234.333333
26,MMP14 (P50281; MMP14_HUMAN). Mmp14 (P53690; MM...,3923.0,4213.0,656,217.666667
10,Ctsl (P06797; CATL1_MOUSE). Ctsl (P07154; CATL...,5273.0,5576.0,644,213.000000
29,MMP9 (P14780; MMP9_HUMAN). Mmp9 (P41245; MMP9_...,4781.0,5112.0,595,197.333333


In [285]:
filename='Proteasix_output.xlsx'
sheetname='diff_ninf'
from openpyxl import load_workbook
with pd.ExcelWriter(filename, engine='openpyxl') as writer:
    writer.book = load_workbook(filename)
    df.to_excel(writer, sheetname)